# 🔍 CICIDS2017 - Data Exploration & Cleaning

**Objectif** : Explorer et nettoyer le dataset CICIDS2017 sur Kaggle

**Dataset** : https://www.kaggle.com/datasets/cicdataset/cicids2017

---

## 📦 Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')


## 📊 Charger le Dataset

Commençons par un seul fichier pour l'exploration

In [ ]:
# Charger un fichier (Monday - trafic normal uniquement)
df = pd.read_csv('/kaggle/input/cicids2017/Monday-WorkingHours.pcap_ISCX.csv')

print(f"Shape: {df.shape}")
print(f"Taille en mémoire: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 🔍 Exploration Initiale

In [ ]:
# Premières lignes
df.head()

In [ ]:
# Informations générales
df.info()

In [ ]:
# Statistiques descriptives
df.describe()

In [ ]:
# Distribution des labels
print("Distribution des labels:")
print(df[' Label'].value_counts())
print(f"\nPourcentages:")
print(df[' Label'].value_counts(normalize=True) * 100)

## 📈 Visualisations

In [ ]:
# Distribution des labels
plt.figure(figsize=(14, 6))
df[' Label'].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Distribution des Labels - CICIDS2017', fontsize=16, fontweight='bold')
plt.xlabel('Type d\'Attaque', fontsize=12)
plt.ylabel('Nombre d\'Instances', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('label_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 🧹 Détection des Problèmes

In [ ]:
# 1. Vérifier les valeurs manquantes
print("=" * 50)
print("1. VALEURS MANQUANTES (NaN)")
print("=" * 50)
nan_counts = df.isnull().sum()
if nan_counts.sum() > 0:
    print(nan_counts[nan_counts > 0])
else:
    print("✅ Aucune valeur NaN détectée")

print(f"\nTotal NaN: {nan_counts.sum()}")

In [ ]:
# 2. Vérifier les valeurs infinies
print("=" * 50)
print("2. VALEURS INFINIES")
print("=" * 50)

numeric_cols = df.select_dtypes(include=[np.number]).columns
inf_counts = {}

for col in numeric_cols:
    inf_count = np.isinf(df[col]).sum()
    if inf_count > 0:
        inf_counts[col] = inf_count

if inf_counts:
    for col, count in sorted(inf_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"{col}: {count}")
else:
    print("✅ Aucune valeur infinie détectée")

print(f"\nTotal colonnes avec infinis: {len(inf_counts)}")

In [ ]:
# 3. Vérifier les duplications
print("=" * 50)
print("3. DUPLICATIONS")
print("=" * 50)
duplicates = df.duplicated().sum()
print(f"Nombre de lignes dupliquées: {duplicates}")
print(f"Pourcentage: {duplicates / len(df) * 100:.2f}%")

In [ ]:
# 4. Vérifier les noms de colonnes
print("=" * 50)
print("4. NOMS DE COLONNES")
print("=" * 50)
print(f"Nombre de colonnes: {len(df.columns)}")
print("\nColonnes avec espaces:")
for col in df.columns:
    if col.startswith(' ') or col.endswith(' '):
        print(f"  - '{col}'")

## 🛠️ Fonction de Nettoyage

In [ ]:
def clean_cicids2017(df):
    """
    Nettoie le dataset CICIDS2017
    
    Args:
        df: DataFrame pandas
    
    Returns:
        DataFrame nettoyé
    """
    print("🧹 Nettoyage en cours...")
    print(f"Shape initiale: {df.shape}")
    
    # 1. Nettoyer les noms de colonnes
    df.columns = df.columns.str.strip().str.replace(' ', '_')
    print("✅ Noms de colonnes nettoyés")
    
    # 2. Supprimer les duplications
    initial_rows = len(df)
    df = df.drop_duplicates()
    duplicates_removed = initial_rows - len(df)
    print(f"✅ Duplications supprimées: {duplicates_removed}")
    
    # 3. Gérer les valeurs infinies
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    print("✅ Valeurs infinies remplacées par NaN")
    
    # 4. Gérer les NaN
    nan_before = df.isnull().sum().sum()
    
    # Remplir avec la médiane pour les colonnes numériques
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df[col].isnull().any():
            df[col].fillna(df[col].median(), inplace=True)
    
    nan_after = df.isnull().sum().sum()
    print(f"✅ NaN traités: {nan_before} → {nan_after}")
    
    # 5. Corriger les valeurs négatives incorrectes
    positive_cols = ['Flow_Duration', 'Total_Fwd_Packets', 'Total_Backward_Packets']
    for col in positive_cols:
        if col in df.columns:
            negative_count = (df[col] < 0).sum()
            if negative_count > 0:
                df.loc[df[col] < 0, col] = 0
                print(f"✅ {col}: {negative_count} valeurs négatives corrigées")
    
    # 6. Supprimer les colonnes non pertinentes pour ML
    cols_to_drop = ['Flow_ID', 'Source_IP', 'Destination_IP', 'Timestamp']
    existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
    if existing_cols_to_drop:
        df = df.drop(columns=existing_cols_to_drop)
        print(f"✅ Colonnes supprimées: {existing_cols_to_drop}")
    
    print(f"\n🎉 Nettoyage terminé!")
    print(f"Shape finale: {df.shape}")
    
    return df

## 🧪 Appliquer le Nettoyage

In [ ]:
# Nettoyer le dataset
df_clean = clean_cicids2017(df.copy())

In [ ]:
# Vérifier le résultat
print("Vérification post-nettoyage:")
print(f"Shape: {df_clean.shape}")
print(f"NaN: {df_clean.isnull().sum().sum()}")
print(f"Infinis: {np.isinf(df_clean.select_dtypes(include=[np.number])).sum().sum()}")
print(f"Duplications: {df_clean.duplicated().sum()}")

## 🏷️ Mapping des Labels

In [ ]:
# Dictionnaire de mapping
ATTACK_CATEGORIES = {
    'BENIGN': 'Normal',
    'FTP-Patator': 'Brute_Force',
    'SSH-Patator': 'Brute_Force',
    'Web Attack – Brute Force': 'Brute_Force',
    'DoS slowloris': 'DoS_DDoS',
    'DoS Slowhttptest': 'DoS_DDoS',
    'DoS Hulk': 'DoS_DDoS',
    'DoS GoldenEye': 'DoS_DDoS',
    'DDoS': 'DoS_DDoS',
    'Web Attack – XSS': 'Web_Attack',
    'Web Attack – SQL Injection': 'Web_Attack',
    'PortScan': 'Reconnaissance',
    'Bot': 'Botnet',
    'Infiltration': 'Advanced_Threat',
    'Heartbleed': 'Advanced_Threat'
}

# Appliquer le mapping
df_clean['Attack_Category'] = df_clean['Label'].map(ATTACK_CATEGORIES)

# Vérifier
print("Distribution des catégories d'attaques:")
print(df_clean['Attack_Category'].value_counts())

In [ ]:
# Encoder numériquement
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_clean['Label_Encoded'] = le.fit_transform(df_clean['Attack_Category'])

# Afficher le mapping
print("Mapping numérique:")
for i, label in enumerate(le.classes_):
    print(f"{i}: {label}")

## 💾 Sauvegarder le Dataset Nettoyé

In [ ]:
# Sauvegarder
output_file = 'cicids2017_monday_cleaned.csv'
df_clean.to_csv(output_file, index=False)
print(f"✅ Dataset sauvegardé: {output_file}")
print(f"Taille: {df_clean.shape}")

## 📊 Traiter Tous les Fichiers (Optionnel)

⚠️ **Attention** : Cela peut prendre beaucoup de temps et de mémoire !

In [ ]:
# Liste de tous les fichiers
files = [
    'Monday-WorkingHours.pcap_ISCX.csv',
    'Tuesday-WorkingHours.pcap_ISCX.csv',
    'Wednesday-workingHours.pcap_ISCX.csv',
    'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
    'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
    'Friday-WorkingHours-Morning.pcap_ISCX.csv',
    'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
    'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'
]

# Traiter tous les fichiers
all_dfs = []

for file in files:
    print(f"\n{'='*60}")
    print(f"Traitement de: {file}")
    print(f"{'='*60}")
    
    try:
        # Charger
        df_temp = pd.read_csv(f'/kaggle/input/cicids2017/{file}')
        
        # Nettoyer
        df_temp_clean = clean_cicids2017(df_temp)
        
        # Mapper les labels
        df_temp_clean['Attack_Category'] = df_temp_clean['Label'].map(ATTACK_CATEGORIES)
        
        # Ajouter à la liste
        all_dfs.append(df_temp_clean)
        
        print(f"✅ {file} traité avec succès")
        
    except Exception as e:
        print(f"❌ Erreur avec {file}: {e}")

# Combiner tous les DataFrames
print(f"\n{'='*60}")
print("Combinaison de tous les fichiers...")
print(f"{'='*60}")

df_final = pd.concat(all_dfs, ignore_index=True)

print(f"\n🎉 Dataset final combiné!")
print(f"Shape: {df_final.shape}")
print(f"\nDistribution finale des labels:")
print(df_final['Attack_Category'].value_counts())

In [ ]:
# Encoder les labels finaux
le_final = LabelEncoder()
df_final['Label_Encoded'] = le_final.fit_transform(df_final['Attack_Category'])

# Sauvegarder le dataset complet
df_final.to_csv('cicids2017_full_cleaned.csv', index=False)
print("✅ Dataset complet sauvegardé: cicids2017_full_cleaned.csv")

## 📥 Prochaines Étapes

1. ✅ Dataset exploré et nettoyé
2. ✅ Labels mappés en catégories
3. ✅ Fichier CSV sauvegardé

**À faire ensuite :**
- Télécharger le fichier depuis Kaggle Output
- Placer dans `data/processed/` de votre projet local
- Commit sur Git
- Passer au Feature Engineering et Training !

---

**Bon courage ! 🚀**